In [2]:
import akshare as ak
import pymysql
from sqlalchemy import create_engine

# 股票代码
stock_code = "000001"

# 获取数据
df = ak.stock_zh_a_hist(
    symbol=stock_code,
    period="daily",
    start_date="20170301",
    end_date="20240528",
    adjust=""
)

# 字段映射
df.rename(columns={
    "日期": "stock_time",
    "开盘": "stock_kp",
    "收盘": "stock_sp",
    "最高": "stock_zg",
    "最低": "stock_zd",
    "成交额": "stock_cje",
    "振幅": "stock_zf",
    "涨跌幅": "stock_zdf",
    "涨跌额": "stock_zde",
    "换手率": "stock_hsl"
}, inplace=True)


In [6]:

# 添加股票代码列
df["stock_id"] = stock_code

# 只保留你数据库中存在的列顺序（避免字段不对）
df = df[[
    "stock_time", "stock_id", "stock_kp", "stock_sp", "stock_zg",
    "stock_zd", "stock_cje", "stock_zf", "stock_zdf", "stock_zde", "stock_hsl"
]]

# 数据库连接信息
db_config = {
    "host": "localhost",
    "port": 3306,
    "user": "root",
    "password": "root",
    "database": "stock",
    "charset": "utf8mb4"
}

# 创建连接引擎
engine_str = f"mysql+pymysql://{db_config['user']}:{db_config['password']}@" \
             f"{db_config['host']}:{db_config['port']}/{db_config['database']}?charset={db_config['charset']}"
engine = create_engine(engine_str)



In [8]:

# 插入数据（追加模式，避免重复插入需配合 UNIQUE 索引）
df.to_sql(name='stock_info', con=engine, if_exists='append', index=False)

print("数据已成功写入 stock_history 表。")

数据已成功写入 stock_history 表。


In [10]:
import akshare as ak

# 获取所有 A 股的代码和名称
stock_df = ak.stock_info_a_code_name()

# 查看前几行
print(stock_df.head())

# 获取股票代码列表
stock_code_list = stock_df['code'].tolist()
print(stock_code_list[:10])  # 打印前10个代码看看


  0%|          | 0/14 [00:00<?, ?it/s]

     code   name
0  000001   平安银行
1  000002  万  科Ａ
2  000004  *ST国华
3  000006   深振业Ａ
4  000007    全新好
['000001', '000002', '000004', '000006', '000007', '000008', '000009', '000010', '000011', '000012']


In [12]:
stock_df

,code,name
0,000001,平安银行
1,000002,万 科Ａ
2,000004,*ST国华
3,000006,深振业Ａ
4,000007,全新好
...,...,...
5407,920445,龙竹科技
5408,920489,佳先股份
5409,920682,球冠电缆
5410,920799,艾融软件


In [16]:
import requests
import re
import time
import random
from PIL import Image
from io import BytesIO

# 生成随机 device_id
def generate_device_id():
    return "e" + "".join(random.choices("0123456789abcdef", k=15))

# 第一步：获取 UUID
def get_uuid():
    url = "https://login.wx.qq.com/jslogin"
    params = {
        "appid": "wx782c26e4c19acffb",
        "fun": "new",
        "lang": "zh_CN",
        "_": int(time.time() * 1000)
    }
    res = requests.get(url, params=params)
    print(f"[DEBUG] jslogin 响应: {res.text}")
    match = re.search(r'"(.*?)"', res.text)
    if match:
        uuid = match.group(1)
        print(f"[+] UUID 获取成功: {uuid}")
        return uuid
    else:
        print("[!] 获取 UUID 失败，响应不符合预期")
        return None

# 第二步：获取二维码
def get_qr(uuid):
    qr_url = f"https://login.weixin.qq.com/qrcode/{uuid}"
    headers = {"User-Agent": "Mozilla/5.0"}
    res = requests.get(qr_url, headers=headers)
    img = Image.open(BytesIO(res.content))
    img.show()
    print("[+] 已弹出二维码，请扫描")

# 第三步：轮询扫码状态
def wait_for_login(uuid):
    if uuid is None:
        print("[!] 无法继续登录，UUID 无效")
        return None

    tip = 1
    login_url = "https://login.wx.qq.com/cgi-bin/mmwebwx-bin/login"
    while True:
        params = {
            "loginicon": "true",
            "uuid": uuid,
            "tip": tip,
            "r": ~int(time.time()),
            "_": int(time.time() * 1000)
        }
        res = requests.get(login_url, params=params)
        print(f"[DEBUG] login 响应: {res.text}")
        code_match = re.search(r"window.code=(\d+);", res.text)
        if not code_match:
            print("[!] 无法解析 login 状态码")
            break

        code = code_match.group(1)

        if code == "201":
            print("[*] 已扫码，请在手机上确认登录")
            tip = 0
        elif code == "200":
            redirect_match = re.search(r'window.redirect_uri="(.*?)";', res.text)
            if redirect_match:
                redirect_uri = redirect_match.group(1) + "&fun=new"
                print(f"[+] 登录成功，跳转链接：{redirect_uri}")
                return redirect_uri
            else:
                print("[!] 登录成功但未获取到 redirect_uri")
                break
        elif code == "408":
            print("[!] 等待扫码中...")
        else:
            print(f"[!] 未知状态码：{code}")
            break
        time.sleep(2)

# 第四步：获取登录凭证
def get_login_info(redirect_uri):
    if redirect_uri is None:
        print("[!] redirect_uri 无效，无法获取登录凭证")
        return

    headers = {"User-Agent": "Mozilla/5.0"}
    res = requests.get(redirect_uri, headers=headers)
    xml = res.text
    print("[DEBUG] 登录 XML 响应:")
    print(xml)

    try:
        skey = re.search(r"<skey>(.*?)</skey>", xml).group(1)
        wxsid = re.search(r"<wxsid>(.*?)</wxsid>", xml).group(1)
        wxuin = re.search(r"<wxuin>(.*?)</wxuin>", xml).group(1)
        pass_ticket = re.search(r"<pass_ticket>(.*?)</pass_ticket>", xml).group(1)

        print("\n🎉 登录凭证获取成功：")
        print("skey:", skey)
        print("wxsid:", wxsid)
        print("wxuin:", wxuin)
        print("pass_ticket:", pass_ticket)
    except AttributeError:
        print("[!] 登录凭证解析失败，请检查 XML 响应格式")

# 主执行流程
def main():
    device_id = generate_device_id()
    uuid = get_uuid()
    if uuid:
        get_qr(uuid)
        redirect_uri = wait_for_login(uuid)
        get_login_info(redirect_uri)

if __name__ == "__main__":
    main()


[DEBUG] jslogin 响应: window.QRLogin.code = 200; window.QRLogin.uuid = "wfOJOFBRVQ==";
[+] UUID 获取成功: wfOJOFBRVQ==
[+] 已弹出二维码，请扫描
[DEBUG] login 响应: window.code=201;window.userAvatar = 'data:img/jpg;base64,/9j/4AAQSkZJRgABAQEASABIAAD/4gIoSUNDX1BST0ZJTEUAAQEAAAIYYXBwbAQAAABtbnRyUkdCIFhZWiAH5gABAAEAAAAAAABhY3NwQVBQTAAAAABBUFBMAAAAAAAAAAAAAAAAAAAAAAAA9tYAAQAAAADTLWFwcGzs/aOOOIVHw220vU962hgvAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAApkZXNjAAAA/AAAADBjcHJ0AAABLAAAAFB3dHB0AAABfAAAABRyWFlaAAABkAAAABRnWFlaAAABpAAAABRiWFlaAAABuAAAABRyVFJDAAABzAAAACBjaGFkAAAB7AAAACxiVFJDAAABzAAAACBnVFJDAAABzAAAACBtbHVjAAAAAAAAAAEAAAAMZW5VUwAAABQAAAAcAEQAaQBzAHAAbABhAHkAIABQADNtbHVjAAAAAAAAAAEAAAAMZW5VUwAAADQAAAAcAEMAbwBwAHkAcgBpAGcAaAB0ACAAQQBwAHAAbABlACAASQBuAGMALgAsACAAMgAwADIAMlhZWiAAAAAAAAD21QABAAAAANMsWFlaIAAAAAAAAIPfAAA9v////7tYWVogAAAAAAAASr8AALE3AAAKuVhZWiAAAAAAAAAoOAAAEQsAAMi5cGFyYQAAAAAAAwAAAAJmZgAA8qcAAA1ZAAAT0AAACltzZjMyAAAAAAABDEIAAAXe///zJgAAB5MAAP2Q///7ov///aMAAAPcAADAbv/bAEMABwUFBgUEBwYGBggHBwgLEgsLC